# Creating first agent

## Install Packages

In [ ]:
%pip install azure-ai-projects==1.0.0b12
%pip install azure-identity
%pip install azure-ai-agents==1.1.0b3

## Import the libraries

In [16]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import MessageRole

## Create the AI Project Client

NOTE: 
- If you don't have Azure CLI, you first need to install it:
   - curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
- Login to Azure
   - az login



In [17]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Create the agent

In [18]:
agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
        name="my-agent",  # Name of the agent
        instructions="You are a helpful agent",  # Instructions for the agent
        
    )

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_p6zy1u1NI6pW8V2fQTHmRpJd


## Create thread

In [19]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_OdOymeHM158u1EuChKtLSQSc


## Helper function
- adds messages to the thread
- run the agent
- display the agent response

In [20]:
def run_agent(user_input):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Check the status of the run and print the result
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        last_msg = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role=MessageRole.AGENT)
        if last_msg:
            print(f"Agent Response: {last_msg.text.value}")

## Run the agent

In [21]:
user_input = "What are the 5 things I can see in New York?"
run_agent(user_input)

Created message, ID: msg_CpSW6rZru2rXw45J0diXLS3M
Run finished with status: completed
Agent Response: Sure! New York City offers a wealth of iconic attractions and experiences. Here are five things you can see and do:

1. **Statue of Liberty**: A symbol of freedom and democracy, this iconic statue is located on Liberty Island and offers stunning views of the Manhattan skyline.

2. **Central Park**: This massive green space in the heart of Manhattan provides a natural escape with walking paths, lakes, and attractions like the Central Park Zoo and the Conservatory Garden.

3. **Times Square**: Known for its bright lights, billboards, and bustling atmosphere, Times Square is a must-visit for its vibrant entertainment options and unique energy.

4. **Empire State Building**: One of the most famous skyscrapers in the world, visitors can go to the observation decks for panoramic views of the city and beyond.

5. **Broadway**: Enjoy world-class theater productions and musicals in the Theater 

## Cleanup resources

In [22]:
# Delete the agent after use
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Deleted agent


## Creating an agent and thread

In [23]:
agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
        name="my-agent",  # Name of the agent
        instructions="You are a helpful agent",  # Instructions for the agent
        
    )
print(f"Created agent, ID: {agent.id}")
agent_id = agent.id


Created agent, ID: asst_WdApzrNxrCcyGroWcAOCLxYG


## Retrieving an agent

In [24]:
agent = project_client.agents.get_agent(agent_id)

## Create thread

To retrieve a thread
- thread = project.agents.threads.get("")

In [25]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_4xsGflvQOb4QjhznCEbFEJya


## Run agent

In [26]:
user_input = "What are the 5 things I can see in San Francisco?"
run_agent(user_input)

Created message, ID: msg_LU7T1n1Z05bJd7yAeqQQPp35
Run finished with status: completed
Agent Response: San Francisco is a vibrant city with a lot to offer. Here are five iconic sights you can see in San Francisco:

1. **Golden Gate Bridge**: One of the most recognizable landmarks in the world, the Golden Gate Bridge offers stunning views and is a must-see for anyone visiting San Francisco.

2. **Alcatraz Island**: This historic island was once a notorious federal prison and is now a popular tourist attraction. You can take a ferry to the island and explore the prison, learning about its history and famous inmates.

3. **Fisherman's Wharf**: A bustling waterfront area with shops, restaurants, and entertainment. You can see sea lions at Pier 39, visit the Aquarium of the Bay, and enjoy fresh seafood.

4. **Chinatown**: The oldest and one of the largest Chinatowns in North America. Walk through the vibrant streets, enjoy authentic Chinese cuisine, and explore unique shops and markets.

5. 

## Cleanup resources

In [27]:
# Delete the agent after use
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Deleted agent
